In [1]:
# @title 🕹️ Interactive Hysteresis Laboratory
# @markdown Run this cell to start the simulator.
# @markdown Browse through the tabs to test the **Bouc-Wen**, **Generalized**, and **Dahl** models.

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output


#Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')

# --- DEFINICIÓN DE LOS MODELOS (EDOs) ---

def input_signal(t, freq=1.0, amp=1.0):
    """Sinusoidal input signal and its derivative"""
    u = amp * np.sin(2 * np.pi * freq * t)
    u_dot = amp * 2 * np.pi * freq * np.cos(2 * np.pi * freq * t)
    return u, u_dot

def bouc_wen_ode(h, t, A, beta, gamma, n, freq, amp):
    u, u_dot = input_signal(t, freq, amp)
    # Ecuación Clásica
    h_dot = A * u_dot - beta * abs(u_dot) * h * abs(h)**(n-1) - gamma * u_dot * abs(h)**n
    return h_dot

def gbw_ode(h, t, A, beta, gamma, n, alpha, freq, amp):
    u, u_dot = input_signal(t, freq, amp)

    # Mantén la estructura Bouc-Wen (disipativa)
    # Asimetría suave: hace beta dependiente del signo de u_dot
    beta_eff = beta * (1.0 + 0.3*np.sign(u_dot))   # 0.3 = nivel de asimetría (ajustable)

    h_dot = alpha * (
        A * u_dot
        - beta_eff * np.abs(u_dot) * h * (np.abs(h)**(n-1))
        - gamma * u_dot * (np.abs(h)**n)
    )
    return h_dot


def dahl_ode(h, t, sigma, hc, i, freq, amp):
    u, u_dot = input_signal(t, freq, amp)
    # Modelo de Dahl
    # Termino de saturación (1 - h/hc * sgn(u_dot))
    bracket = 1 - (h / hc) * np.sign(u_dot)
    # Evitar números complejos si el corchete es negativo (estabilidad numérica)
    bracket = np.maximum(bracket, 0)
    h_dot = sigma * u_dot * (bracket**i)
    return h_dot

# --- LÓGICA DE SIMULACIÓN Y GRÁFICOS ---

def run_simulation(model_type, params):
    # Tiempo de simulación
    t = np.linspace(0, 5, 1000) # 5 segundos

    # Resolver EDO según el modelo seleccionado
    h0 = 0 # Condición inicial

    if model_type == 'CBW':
        sol = odeint(bouc_wen_ode, h0, t, args=(params['A'], params['beta'], params['gamma'], params['n'], params['freq'], params['amp']))
    elif model_type == 'GBW':
        sol = odeint(gbw_ode, h0, t, args=(params['A'], params['beta'], params['gamma'], params['n'], params['alpha'], params['freq'], params['amp']))
    elif model_type == 'Dahl':
        sol = odeint(dahl_ode, h0, t, args=(params['sigma'], params['hc'], params['i'], params['freq'], params['amp']))

    h = sol[:, 0]
    u, _ = input_signal(t, params['freq'], params['amp'])

    # --- PLOTTING ---
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Gráfica 1: Ciclo de Histéresis (Plano de Fase)
    ax1.plot(u, h, 'b-', linewidth=2, label='Histéresis')
    ax1.plot(u, u, 'k--', alpha=0.3, label='Lineal (Ref)')
    ax1.set_title(f'Hysteresis Loop ({model_type})', fontsize=14, fontweight='bold', color='navy')
    ax1.set_xlabel('Input Voltage $u(t)$ [V]')
    ax1.set_ylabel('Displacement $h(t)$ [um]')
    ax1.legend()
    ax1.grid(True, linestyle='--', alpha=0.7)

    # Gráfica 2: Series de Tiempo
    ax2.plot(t, u, 'k--', label='Input $u(t)$', alpha=0.6)
    ax2.plot(t, h, 'r-', linewidth=2, label='Output $h(t)$')
    ax2.set_title('Time Response', fontsize=14, fontweight='bold', color='navy')
    ax2.set_xlabel('Time [s]')
    ax2.set_ylabel('Amplitude')
    ax2.legend()
    ax2.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()

# --- INTERFAZ GRÁFICA (WIDGETS) ---

# 1. Widgets para Classical Bouc-Wen
style = {'description_width': 'initial'}
layout = widgets.Layout(width='95%')

cbw_ui = widgets.VBox([
    widgets.Label(value="📐 Classical Model Parameters (Symmetric):"),
    widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='A (Amplitude)', style=style, layout=layout),
    widgets.FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, description='β (Beta - Shape)', style=style, layout=layout),
    widgets.FloatSlider(value=0.5, min=-1.0, max=1.0, step=0.1, description='γ (Gamma - Shape)', style=style, layout=layout),
    widgets.FloatSlider(value=1.0, min=1.0, max=5.0, step=0.5, description='n (Smoothness)', style=style, layout=layout),
    widgets.HTML(value="<hr>"),
    widgets.Label(value="🔌 Input Signal::"),
    widgets.FloatSlider(value=1.0, min=0.5, max=10.0, step=0.5, description='Input Amplitude', style=style, layout=layout)
])

def update_cbw(A, beta, gamma, n, amp):
    params = {'A':A, 'beta':beta, 'gamma':gamma, 'n':n, 'freq':1.0, 'amp':amp}
    run_simulation('CBW', params)

cbw_out = widgets.interactive_output(update_cbw,
                                     {'A': cbw_ui.children[1], 'beta': cbw_ui.children[2],
                                      'gamma': cbw_ui.children[3], 'n': cbw_ui.children[4],
                                      'amp': cbw_ui.children[7]})

# 2. Widgets para Generalized Bouc-Wen
gbw_ui = widgets.VBox([
    widgets.Label(value="⚖️ Generalized Parameters (Asymmetry):"),
    widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='α (Alpha - Scale)', style=style, layout=layout),
    widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='A (Amplitude)', style=style, layout=layout),
    widgets.FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, description='β (Beta - Asymmetry 1)', style=style, layout=layout),
    widgets.FloatSlider(value=0.5, min=-1.0, max=1.0, step=0.1, description='γ (Gamma - Asymmetry 2)', style=style, layout=layout),
    widgets.FloatSlider(value=1.0, min=1.0, max=5.0, step=0.5, description='n (Smoothness)', style=style, layout=layout),
    widgets.HTML(value="<br><b>Note:</b> If β and γ differ, the loop will deform."),
    widgets.HTML(value="<hr>"),
    widgets.FloatSlider(value=1.0, min=0.5, max=10.0, step=0.5, description='Input Amplitude', style=style, layout=layout)
])

def update_gbw(alpha, A, beta, gamma, n, amp):
    params = {'alpha':alpha, 'A':A, 'beta':beta, 'gamma':gamma, 'n':n, 'freq':1.0, 'amp':amp}
    run_simulation('GBW', params)

gbw_out = widgets.interactive_output(update_gbw,
                                     {'alpha': gbw_ui.children[1], 'A': gbw_ui.children[2],
                                      'beta': gbw_ui.children[3], 'gamma': gbw_ui.children[4],
                                      'n': gbw_ui.children[5], 'amp': gbw_ui.children[8]})

# 3. Widgets para Dahl Model
dahl_ui = widgets.VBox([
    widgets.Label(value="🧱 Dahl Model Parameters (Solid Friction):"),
    widgets.FloatSlider(value=2.0, min=0.1, max=10.0, step=0.1, description='σ (Sigma - Initial Stiffness)', style=style, layout=layout),
    widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='hc (Saturation Force)', style=style, layout=layout),
    widgets.FloatSlider(value=1.0, min=0.5, max=5.0, step=0.1, description='i (Shape Factor)', style=style, layout=layout),
    widgets.HTML(value="<br><b>Tip:</b> Aumenta 'i' para hacer las esquinas más cuadradas (plastic transition)."),
    widgets.HTML(value="<hr>"),
    widgets.FloatSlider(value=1.5, min=0.5, max=10.0, step=0.5, description='Input Amplitude', style=style, layout=layout)
])

def update_dahl(sigma, hc, i, amp):
    params = {'sigma':sigma, 'hc':hc, 'i':i, 'freq':1.0, 'amp':amp}
    run_simulation('Dahl', params)

dahl_out = widgets.interactive_output(update_dahl,
                                      {'sigma': dahl_ui.children[1], 'hc': dahl_ui.children[2],
                                       'i': dahl_ui.children[3], 'amp': dahl_ui.children[6]})

# --- PESTAÑAS (TABS) ---
tab = widgets.Tab(children=[
    widgets.VBox([cbw_ui, cbw_out]),
    widgets.VBox([gbw_ui, gbw_out]),
    widgets.VBox([dahl_ui, dahl_out])
])
tab.set_title(0, '1. Classical Bouc-Wen')
tab.set_title(1, '2. Generalized Bouc-Wen')
tab.set_title(2, '3. Dahl Model')

display(tab)